# Capital Analysis

This notebook analyzes the company's capital structure and computes key metrics
such as WACC, cost of equity, cost of debt, and capital efficiency ratios.

In [1]:
from src.setup import * 

## Step 1: Define Capital Analysis Function

This function calculates capital structure ratios, financing costs, and the
weighted average cost of capital (WACC) using balance sheet and income statement data.

In [2]:

def capital_analysis(df, balance_sheet, financials, market_cap, risk_free_rate, beta,
                     market_risk_premium, credit_spread, tax_rate, ticker, interest_expense=None):
    """
    Compute capital structure and WACC metrics.

    Parameters
    ----------
    df : pandas.DataFrame
        Quarterly balance sheet data.
    balance_sheet : pandas.DataFrame
        Latest balance sheet data.
    financials : pandas.DataFrame
        Yahoo Finance financials data.
    market_cap : float
        Current market capitalization of the company.
    risk_free_rate : float
        Risk-free interest rate (as decimal or percent depending on usage).
    beta : float
        Stock beta for CAPM calculation.
    market_risk_premium : float
        Expected market risk premium.
    credit_spread : float
        Credit spread for debt cost calculation.
    tax_rate : float
        Corporate tax rate in percent.
    ticker : str
        Stock ticker symbol.
    interest_expense : float, optional
        Interest expense value; if None, taken from financials.

    Returns
    -------
    tuple
        df_results : pandas.DataFrame
            Formatted table of calculated metrics.
        results : dict
            Dictionary with numeric metric values.
    """
    
    # --- Get latest financial values safely ---
    equity = df[f"{ticker}: Quarterly Shareholders Equity"].iloc[-1]
    debt = balance_sheet.loc["Total Debt"].iloc[0]
    revenue = financials.loc["Total Revenue"].iloc[0]
    ebit = financials.loc["EBIT"].iloc[0]
    
    # Use provided interest expense or fallback to financials
    if interest_expense is None:
        interest_expense = financials.loc["Interest Expense"].iloc[0] 
    
     # Ratios for WACC calculation
    total_assets = debt + equity
    equity_ratio = equity / total_assets if total_assets != 0 else np.nan
    debt_ratio = debt / total_assets if total_assets != 0 else np.nan
    
    equity_ratio_wacc = market_cap / (market_cap + debt) if (market_cap + debt) != 0 else np.nan
    debt_ratio_wacc = debt / (market_cap + debt) if (market_cap + debt) != 0 else np.nan
    
    # --- Cost calculations ---
    cost_of_debt = (risk_free_rate + credit_spread) * (1 - tax_rate / 100)
    cost_of_equity = risk_free_rate + beta * market_risk_premium
    wacc = equity_ratio_wacc * cost_of_equity + debt_ratio_wacc * cost_of_debt
    
    # --- Other metrics ---
    revenue_to_capital = revenue / total_assets if total_assets != 0 else np.nan
    interest_coverage = ebit / interest_expense if interest_expense != 0 else np.nan
    
    results = {
        "Total Assets": total_assets,
        "Debt": debt,
        "Equity": equity,
        "Equity Ratio": equity_ratio,
        "Debt Ratio": debt_ratio,
        "Cost of Equity": cost_of_equity,
        "Cost of Debt": cost_of_debt,
        "WACC": wacc,
        "Revenue to Capital": revenue_to_capital,
        "Interest Coverage Ratio": interest_coverage
    }
    
    # --- Convert to DataFrame for nice display ---
    df_results = pd.DataFrame(results, index=[0]).T.rename(columns={0: "Value"})
    df_results["Value"] = df_results["Value"].apply(lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x)
    
    return df_results, results




## Step 2: Run Capital Analysis

We apply the capital analysis function to compute the company’s financing structure
and cost of capital.


In [3]:
# --- Example usage ---
capital_df, capital_dict = capital_analysis(
    df, balance_sheet, info.financials, market_cap=market_cap,
    risk_free_rate=RISK_FREE_RATE, beta=BETA,
    market_risk_premium = market_risk_premium,
    credit_spread=CREDIT_SPREAD, tax_rate=TAX_RATE, ticker=TICKER
)

## Step 3: Results

The table below summarizes the company’s capital structure and cost metrics.

In [4]:
display(capital_df)  # Display formatted table

,Value
Total Assets,"423,664,000,000.00"
Debt,"60,588,000,000.00"
Equity,"363,076,000,000.00"
Equity Ratio,0.86
Debt Ratio,0.14
Cost of Equity,0.09
Cost of Debt,0.03
WACC,0.09
Revenue to Capital,0.66
Interest Coverage Ratio,52.84
